In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import joblib
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_squared_error


In [ ]:
class DataLoader:
    def __init__(self, train_path, val_path, test_path, drop_columns=None, target_cols=None):
        self.train_path = train_path
        self.val_path = val_path
        self.test_path = test_path
        self.drop_columns = drop_columns or ['batch_size']
        self.target_cols = target_cols or ['runtime_sec', 'powerty_consumption']

    def _load_split(self, path):
        df = pd.read_excel(path)

        # Удалим ненужные столбцы, если они есть
        for col in self.drop_columns:
            if col in df.columns:
                df = df.drop(columns=col)

        X = df.drop(columns=self.target_cols)
        y = df[self.target_cols]
        return X, y

    def load(self):
        X_train, y_train = self._load_split(self.train_path)
        X_val, y_val = self._load_split(self.val_path)
        X_test, y_test = self._load_split(self.test_path)

        return X_train, y_train, X_val, y_val, X_test, y_test


In [ ]:
data_loader = DataLoader(
    train_path='train_set.xlsx',
    val_path='validation_set.xlsx',
    test_path='test_set.xlsx'
)

X_train, y_train, X_val, y_val, X_test, y_test = data_loader.load()


In [ ]:

# Обучение модели для runtime_sec
model_runtime = lgb.LGBMRegressor(random_state=42)
model_runtime.fit(X_train, y_train['runtime_sec'])

# Обучение модели для powerty_consumption
model_power = lgb.LGBMRegressor(random_state=42)
model_power.fit(X_train, y_train['powerty_consumption'])

# Предсказания на валидации
pred_runtime_val = model_runtime.predict(X_val)
pred_power_val = model_power.predict(X_val)

# Метрики
mse_runtime = mean_squared_error(y_val['runtime_sec'], pred_runtime_val)
mse_power = mean_squared_error(y_val['powerty_consumption'], pred_power_val)

print(f"MSE runtime_sec: {mse_runtime:.6f}")
print(f"MSE powerty_consumption: {mse_power:.6f}")


In [ ]:
# Предсказания на тестовом наборе
pred_runtime_test = model_runtime.predict(X_test)
pred_power_test = model_power.predict(X_test)

# Сохраняем предсказания вместе с реальными значениями
results_test = pd.DataFrame({
    'runtime_actual': y_test['runtime_sec'],
    'runtime_pred': pred_runtime_test,
    'power_actual': y_test['powerty_consumption'],
    'power_pred': pred_power_test
})

results_test.to_excel('test_predictions.xlsx', index=False)

# Сохраняем модели
joblib.dump(model_runtime, 'model_runtime.pkl')
joblib.dump(model_power, 'model_power.pkl')
